Linear Regression
1. Preprocess Test data and get predictions
2. Compute Mean Abolute Error, Mean Square error for test data
3. Implement Ridge and Lasso Regression and then compute the following metrics on test data

Trees
Compute errors on test sets
Play with different parameter of decision trees and random forests and see the impact on train and test error
[OPTIONAL] implement cross validation and get best hyperparameters